# Lab Class ML:II

# Ankit Satpute, 120825 ; Hsueh Wei, 120820; Sagar Nagaraj Simha, 120797 -(M.Sc. CS4DM) - Group 13

## Exercise 3 : Concept Learning

In [1]:
import numpy as np

## 3a

In [2]:
# '#' is used for denoting maximally specific hypothesis
def g_0(n):
    return ("?",) * n

def s_0(n):
    return ("#",) * n

## 3b

In [3]:
# Returns True if h1 >=g h2
def more_general(h1, h2):
    isgeneral = []
    for i in range(len(h1)):
        if h1[i]=='?':
            isgeneral.append(True)
        elif h1[i]!='#':
            if (h1[i]==h2[i] or h2[i]=='#'):
                isgeneral.append(True)
            else:
                isgeneral.append(False)
        else:
            isgeneral.append(False)
    return all(isgeneral)

In [4]:
more_general(("sunny", "warm", "?"), ("sunny", "?", "normal"))

False

## 3c

In [5]:
# Returns the minimum generalizations of the given hypothesis h w.r.t the current training data
def min_generalizations(h, x):
    h_mingen =[]
    for fea in range(0,len(h)):
        if h[fea]=="#":
            h_mingen.append(x[fea])
        else:
            if h[fea]==x[fea]:
                h_mingen.append(h[fea])
            else:
                h_mingen.append("?")
    return tuple(h_mingen)

In [6]:
min_generalizations(h=("sunny", "warm"), x=("rainy", "warm"))

('?', 'warm')

## 3d

In [7]:
# return all minimal specializations of hypothesis h with respect to domains and the training data x
def min_specializations(h, domains, x):
    h_minspec = []
    for fea in range(len(h)):
        if h[fea] =="?":
            for i in range(len(domains[fea])):
                if domains[fea][i] == x[fea]:
                    continue # Pick next value in domain
                else:
                    h_minspec.append( h[:fea]+ (domains[fea][i],) + h[fea+1:] )
        elif h[fea] !="#":
            h_minspec.append( h[:fea]+ ("#",) +h[fea+1:] )
    return h_minspec

In [8]:
minspec = min_specializations(h=('?', 'x',), 
                    domains=[['a', 'b', 'c'], ['x', 'y']], 
                    x=('b', 'x'))
minspec

[('a', 'x'), ('c', 'x'), ('?', '#')]

# 3e - Candidate-Elimination algorithm

In [9]:
def p_check_g(x, G):
    
    #FOREACH g ∈ G DO IF g(x) ?= 1 THEN G = G \ {g} ENDDO   
    Gnew = G.copy()
    for h in Gnew:
        for feature in range(len(h)):
            if h[feature] == "?":
                continue
            elif (h[feature] != x[feature]  or h[feature]=='#'):
                #Remove h from G
                G.remove(h)
                break
    return G

In [10]:
def p_check_s(x, G, S):

    #FOREACH s ∈ S DO IF s(x) ?= 1 THEN S = S \ {s} ENDDO
    S_plus = []
    Snew = S.copy()
    for h in Snew:
        S_plus.clear()
        for feature in range(len(h)):
            if h[feature] == "?":
                continue
            elif (h[feature] != x[feature]  or h[feature]=='#'):
                #Remove h from S
                S.remove(h)
                break
                
        S_plus.append(min_generalizations(h,x))
        #print('Splus',len(S_plus))
        
        #Adding s which are less general than g in G
        if S_plus!= None:
            for s in S_plus:
                for g in G:
                    if more_general(g,s):
                        S.append(s)
                        
        # Removing s which are more general
        remove_list = []
        
        for i in range(len(S)):
            for j in range(len(S)):
                if S[i] == S[j]:
                    continue
                if more_general(S[i],S[j]):
                    remove_list.append(i)
                    break
        for index in sorted(remove_list, reverse=True):
            del S[index]
    return S

In [11]:
def n_check_s(x, S):
    
    #FOREACH s ∈ S DO IF s(x) ?= 0 THEN S = S \ {s} ENDDO
    Snew = S.copy()
    for h in Snew:
        is_remove = True
        for feature in range(len(h)):
            if h[feature] == "?":
                continue
            elif (h[feature] != x[feature])  or (h[feature]=='#'):
                is_remove = False
        # s(x) = 1
        if is_remove:
            S.remove(h)
    return S

In [12]:
def n_check_g(x, domains, G, S):
    
    #FOREACH g ∈ G DO, IF g(x) ?= 0 THEN G = G \ {g}, G− = min_specializations(g, x)
    G_minus = []
    Gnew = G.copy()
    for h in Gnew:
        G_minus.clear()
        is_remove = True
        for feature in range(len(h)):
            if h[feature] == "?":
                continue
            elif (h[feature] != x[feature])  or (h[feature]=='#'):
                is_remove = False        
        # s(x) = 1
        if is_remove:
            G.remove(h)
            minspec = min_specializations(h,domains,x)
            for each in minspec:
                G_minus.append(each)
                
        #Adding g which are more general than s in S
        if G_minus!= None:
            for g in G_minus:
                for s in S:
                    if more_general(g,s):
                        G.append(g)
        
        # Removing g which are NOT more general
        remove_list = []
        for i in range(len(G)):
            for j in range(len(G)):
                if G[i] == G[j]:
                    continue
                if more_general(G[i],G[j]):
                    remove_list.append(j)
                    break
        for index in sorted(remove_list, reverse=True):
            del G[index]
    return G

In [13]:
# Candidate Elimination algorithm
def candidate_elimination(examples):
    domains = []
    for i in range(len(examples[0])):
        attr = []
        for each_instance in examples:
            attr.append(each_instance[i])
        domains.append(list(set(attr)))

    #Maximally generalized 
    G = []
    G.append(g_0(len(examples[0])-1))
    #Maximally specialized
    S = []
    S.append(s_0(len(examples[0])-1))


    for each in examples:
        if each[len(examples[0])-1] == "True":
            G = p_check_g(each[:-1],G)
            S = p_check_s(each[:-1],G,S)
        elif each[len(examples[0])-1] == "False":
            S = n_check_s(each[:-1],S)
            G = n_check_g(each[:-1],domains, G, S)

    return (tuple(set(G)),tuple(set(S)))

## 3f - EnjoySport Task. Returns G and S

In [14]:
#Examples in the form of list of (n + 1)-tuples
examples = [('sunny', 'warm', 'normal', 'strong', 'warm', 'same', 'long', 'True'),
 ('sunny', 'warm', 'high', 'strong', 'warm', 'same', 'long', 'True'),
 ('rainy', 'cold', 'high', 'weak', 'warm', 'change', 'medium', 'False'),
 ('sunny', 'warm', 'high', 'strong', 'cool', 'change', 'long', 'True'),
('cloudy', 'cold' ,'high' ,'strong' ,'cool', 'change', 'short' ,'False')]

# examples = [('monday','no','easygoing','evening','True'),('monday','no','annoyed','evening','False'),
#            ('saturday','yes','easygoing','lunchtime','False'),('monday','no','easygoing','morning','True')]

G,S = candidate_elimination(examples)
G = list(set(G))
print("G is",G)
print('\n')
S = list(set(S))
print("S is",S)

G is [('sunny', '?', '?', '?', '?', '?', '?'), ('sunny', '?', '?', 'strong', '?', '?', '?'), ('?', 'warm', '?', '?', '?', '?', '?'), ('?', '?', '?', 'strong', '?', '?', 'long'), ('?', 'warm', '?', 'strong', '?', '?', '?'), ('?', '?', '?', '?', '?', '?', 'long')]


S is [('sunny', 'warm', '?', 'strong', '?', '?', 'long')]


## 3g - Hypothesis space V H,D spanned by G and S

In [15]:
version_space=[]
for g in G:
    for s in S:
        for i in range(len(g)):
            if g[i] == '?' and s[i] != '?':
                version_space.append((g[:i] + (s[i],) + g[i+1:]))
versionspace_list = list(set(version_space))
Boundaries = G+S
versionspace = list(set(versionspace_list).difference(Boundaries))
#versionspace
print('Version Space is:')
print('S :\n', S,'\n')
print('Hypotheses in between S and G -->\n', versionspace, '\n')
print('G -->\n', G, '\n')

Version Space is:
S :
 [('sunny', 'warm', '?', 'strong', '?', '?', 'long')] 

Hypotheses in between S and G -->
 [('sunny', 'warm', '?', 'strong', '?', '?', '?'), ('sunny', 'warm', '?', '?', '?', '?', '?'), ('?', 'warm', '?', '?', '?', '?', 'long'), ('?', 'warm', '?', 'strong', '?', '?', 'long'), ('sunny', '?', '?', 'strong', '?', '?', 'long'), ('sunny', '?', '?', '?', '?', '?', 'long')] 

G -->
 [('sunny', '?', '?', '?', '?', '?', '?'), ('sunny', '?', '?', 'strong', '?', '?', '?'), ('?', 'warm', '?', '?', '?', '?', '?'), ('?', '?', '?', 'strong', '?', '?', 'long'), ('?', 'warm', '?', 'strong', '?', '?', '?'), ('?', '?', '?', '?', '?', '?', 'long')] 

